# Notebook para manutenção do código

In [ ]:
import datetime
import seaborn as sns
import numpy as np
import pandas as pd
import yfinance as yf
import local_lib as lib

## local_lib

In [ ]:
#@st.cache
def etl_tesouro_historic_price():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df = pd.read_csv(url, sep=';', decimal=',')
    df['data'] = pd.to_datetime(df['Data Base'], format='%d/%m/%Y')
    df['ticker'] = df['Tipo Titulo'].astype(str) + ' ' + df['Data Vencimento'].str[6:]
    df['preco_hist'] = round(df['PU Base Manha'].astype(float), 2)
    return df

#-----------------------------------------------------------------------------------------------------
#@st.cache  
def etl_benchmark_historic_price():
    # CDI
    df_cdi = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json')
    df_cdi['data'] = pd.to_datetime(df_cdi['data'], format='%d/%m/%Y')
    df_cdi.columns = ['data','cdi']

    # IPCA
    df_ipca = pd.read_json('http://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json')
    df_ipca['data'] = pd.to_datetime(df_ipca['data'], format='%d/%m/%Y')
    df_ipca.columns = ['data','ipca']
    df_ipca['ipca'] = round((1 + df_ipca['ipca']) ** (1/22) - 1, 6)

    # IBOV
    df_ibov = yf.download('^BVSP', interval='1d')['Adj Close'].reset_index(drop=False)
    df_ibov.columns = ['data','ibov']
    df_ibov['ibov'] = ((df_ibov['ibov'] / df_ibov['ibov'].shift(1) - 1) * 100).fillna(0).round(6)
        
    # S&P500
    df_sp500 = yf.download('^GSPC', interval='1d')['Adj Close'].reset_index(drop=False)
    df_sp500.columns = ['data','sp500']
    df_sp500['sp500'] = ((df_sp500['sp500'] / df_sp500['sp500'].shift(1) - 1) * 100).fillna(5).round(6)

    df_final = pd.merge(df_ibov, df_sp500, on='data', how='inner')
    df_final = pd.merge(df_final, df_cdi, on='data', how='inner')
    df_final = pd.merge(df_final, df_ipca, on='data', how='left')
    df_final['ipca'] = df_final['ipca'].fillna(method='ffill')
    df_final['data'] = pd.to_datetime(df_final['data'])

    return df_final


# ------------------------------------------------------------------------------------------------------
#@st.cache
def etl_bolsa_historic_price(list_ticker_b3: list, start_date: str, end_date: str) -> np.array:

    # Utilizando a api do yf
    long_string = ' '.join([i + '.SA' for i in list_ticker_b3])
    df_price = yf.download(long_string, start=start_date, end=end_date, group_by='column')['Adj Close'].reset_index()

    # Ajustes na base
    df_price.columns = ['data'] + list(list_ticker_b3)    
    df_price['data'] = pd.to_datetime(df_price['data'])
    df_price = df_price.fillna(0).round(2)
    
    return df_price

## Home

In [ ]:
import datetime
import json
import numpy as np
import pandas as pd
from PIL import Image
import os
import streamlit as st

In [ ]:
# Extra script
uploaded_files = [
    '../data/kenji/movimentacao-2019.xlsx',
    '../data/kenji/movimentacao-2020.xlsx',
    '../data/kenji/movimentacao-2021.xlsx',
    '../data/kenji/movimentacao-2022.xlsx',
]

In [ ]:
# Extração.
for i in uploaded_files:
    df = pd.read_excel(i, engine='openpyxl')

    # No primeiro caso criaremos um dataframe que consolidará todas as movimentações.
    if i == uploaded_files[0]:
        df_all = df
    else:
        df_all = pd.concat([df_all, df], axis=0, ignore_index=True).drop_duplicates(keep='last')

df_all.head()

In [ ]:
# Tratamentos:
# (geral) Nome e dtype.
dict_dtype = {'credito_ou_debito': str,
                'data': str,
                'tp_movimento': str,
                'ativo': str,
                'instituicao': str,
                'qt_abs': float,
                'preco_mov': float,
                'vl_total_abs': float}

df_all.columns = list(dict_dtype.keys())
df_all['preco_mov'].replace('-', 0, inplace=True)
df_all['vl_total_abs'].replace('-', 0, inplace=True)
df_all = df_all.astype(dict_dtype)
df_all['data'] = pd.to_datetime(df_all['data'], format='%d/%m/%Y')
df_all.head()

In [ ]:
# Check (apagar)
df_all.loc[df_all['ativo'].str.contains('B3SA3')].sort_values('data')
#df_all.loc[df_all['tp_movimento'].str.contains('Desdobro')].sort_values('data')

In [ ]:
# (geral) Nova variável: classificação do ativo.
df_all['tp_ativo'] = np.select(
    [
        (df_all['ativo'].str.upper()).str.contains('TESOURO'),
        df_all['ativo'].str.split(' - ', 0).str[0].str.len() == 5,
        df_all['ativo'].str.split(' - ', 0).str[0].str.len() == 6,
        df_all['ativo'].str.contains('DEB'),
        df_all['ativo'].str.contains('|'.join(['CDB', 'RDB', 'LCA', 'LCI']))
    ],
    [
        'Tipo 1: tesouro',
        'Tipo 2: ações',
        'Tipo 3: BDR',
        'Tipo 4: debêntures',
        'Tipo 5: renda fixa privada'
    ],'?'
)

# (geral) Nova variável: ticker.
df_all['ticker'] = np.select(
    [
        df_all['tp_ativo'] == 'Tipo 4: debêntures',
        df_all['tp_ativo'] == 'Tipo 5: renda fixa privada'
    ],
    [   
        df_all['ativo'].str[5:12],
        df_all['ativo'].str[5:17]
    ], df_all['ativo'].str.split(' - ').str[0]
)

# (bolsa) Ajuste específico de ações: zerar a quantidade de compra/venda em caso de dividendos e juros sobre capital próprio.
df_all['qt_abs'] = np.where(df_all['tp_movimento'].isin(['Transferência - Liquidação', 'Bonificação em Ativos', 'Desdobro']), df_all['qt_abs'], 0)

# (geral) Nova variável: variação na quantidade de ativos.
df_all['qt'] = df_all['qt_abs'] * np.where(df_all['credito_ou_debito'] == 'Credito', 1, -1)

# (geral) Nova variável: variação na quantidade no valor total.
df_all['vl_total'] = df_all['vl_total_abs'] * np.where(df_all['credito_ou_debito'] == 'Credito', 1, -1)

# (bolsa) Nova variável: flag se a negociação é um provento (dividendo, juros sobre capital próprio ou leilão)
df_all['provento'] = np.where(df_all['tp_movimento'].isin(['Dividendo', 'Juros Sobre Capital Próprio', 'Fração em Ativos', 'Leilão de Fração']), 'provento', 'negociacao')

df_all.head()

In [ ]:
#Check (apagar)

a =df_all.loc[df_all['ticker'] == 'ITSA4'].sort_values('data')
a#['provento'] = np.where(a['tp_movimento'].isin(['Dividendo', 'Juros Sobre Capital Próprio', 'Fração em Ativos', 'Leilão de Fração']), 1, 0)
#a['provento'].value_counts()

#a['negociacao_ou_provento'] = np.where(a['tp_movimento'].str.isin(['Transferência - Liquidação']))
#a['qt_abs'] = np.where(a['tp_movimento'].isin(['Transferência - Liquidação', 'Bonificação em Ativos']), a['qt_abs'], 0)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#    display(a)

In [ ]:
# (f) Unificar múltiplas compras/vendas em diferentes corretoras.
# Essa etapa necessariamente é a última, pois aplicaremos cálculo sobre quantidade e preço.
# Ao final, teremos o preço médio de compras/venda 
# tp_movimento foi removido pois podem haver compras e vendas o mesmo dia (caso de daytrade)
#df_all = df_all.groupby(['tp_ativo','ticker','data']).agg({'qt':'sum', 'vl_total':'sum'}).reset_index(drop=False)
#df_all['preco_mov'] = np.where(df_all['qt'] != 0, round(df_all['vl_total'] / df_all['qt'], 2), 0)

In [ ]:
# Futuro: criar uma visão geral de todos os investimento.s

In [ ]:
# Filtrar apenas Tesouro para a página seguinte.
def only_tesouro(df):
    df = df.groupby(['tp_ativo','ticker','data']).agg({'qt':'sum', 'vl_total':'sum'}).reset_index(drop=False)
    df['preco_mov'] = np.where(df['qt'] != 0, round(df['vl_total'] / df['qt'], 2), 0)
    df = df.loc[df['tp_ativo'] == 'Tipo 1: tesouro'].sort_values(by=['ticker','data'], ascending=True)
    
    return df[['data', 'ticker', 'qt', 'preco_mov', 'vl_total']]

df_tesouro = only_tesouro(df_all)
df_tesouro.head()

In [ ]:
# Filtrar ações e BDR para a página seguinte.
def only_bolsa(df):
    df = df.loc[(df['tp_ativo'] == 'Tipo 2: ações') | (df['tp_ativo'] == 'Tipo 3: BDR')].sort_values(by=['ticker','data'], ascending=True)
    df = df[['data', 'ticker', 'tp_movimento', 'provento', 'qt', 'preco_mov', 'vl_total']]
    df = df.groupby(['data', 'ticker','provento']).agg({'qt':'sum', 'vl_total':'sum'}).reset_index(drop=False)
    return df
    
df_bolsa = only_bolsa(df_all)
df_bolsa.head()

In [ ]:
# Check (apagar)
a = df_bolsa.loc[df_bolsa['ticker'] == 'ITSA4'].sort_values('data')
a

## Tesouro Direto

In [ ]:
import pandas as pd
import yfinance as yf
import local_lib as lib

### Parte 1: Tabela Dinâmica

In [ ]:
# Carregando os dados inputados pelo usuário.
df_tesouro = pd.read_csv('../data/manutencao/dados_pos_home.csv')
df_tesouro['data'] = pd.to_datetime(df_tesouro['data'], format='%Y-%m-%d')
df_tesouro.head()

In [ ]:
# Carregando os preços diários extraídos da API do Tesouro.
df_hist_tesouro = etl_tesouro_historic_price()
df_hist_tesouro.head()

In [ ]:
# Juntando os dados do usuário com os dados da API.
df_tesouro_historico = lib.merge_historic_tesouro(df_hist_tesouro, df_tesouro)
df_tesouro_historico.head()

In [ ]:
# Passo extra para criar uma coluna identificando o último dia do mês de cada ativo (útil para os plots)
df_tesouro_historico = lib.create_column_last_day(df_tesouro_historico)
df_tesouro_historico.head()

In [ ]:
# Selecionar os tickers a serem visualizados (desnecessário neste notebook)
'''
# Filtro de tickers.
list_ticker = st.multiselect('Escolha o(s) investimento(s):',
                                df_tesouro_historico['ticker'].unique().tolist(),
                                df_tesouro_historico['ticker'].unique().tolist())
df_tesouro_historico = df_tesouro_historico.loc[df_tesouro_historico['ticker'].isin(list_ticker)]
'''

In [ ]:
# Preparar os dados exclusivamente para a Tabela Dinâmica (visão 1)
df_plot = df_tesouro_historico.loc[df_tesouro_historico['dummy_ultimo_dia'] == 1]
tab1, data_col = lib.custom_pivot_table(df_plot, col_value='vl_atualizado')
tab1

### Parte 2: Gráfico de Linha com Benchmark

In [ ]:
# Extração das variações dos benchmarks (API do BC e yfinance)
df_hist_bench = etl_benchmark_historic_price()
df_hist_bench.head()

In [ ]:
# Junção dos dados inputados pelo usuário e os dados históricos do benchmarks
df_tesouro_historico_agg = lib.merge_historic_benchmark(df_tesouro_historico, df_hist_bench)
df_tesouro_historico_agg.head()

In [ ]:
# Arrumando dados em um formato melhor para o gráfico.
tab2 = lib.custom_data_lineplot(df_tesouro_historico_agg, ['ibov', 'sp500', 'cdi', 'ipca'])
tab2.head()

### Parte 3: KPI

In [ ]:
date_interval = (datetime.datetime(2022, 7, 3), datetime.datetime(2022, 12, 1))
date_interval

In [ ]:
df_date = pd.DataFrame({'data': pd.date_range(df_tesouro_historico['data'].min().to_pydatetime() - datetime.timedelta(days=1),
                                                df_tesouro_historico['data'].max().to_pydatetime())})
df_kpi = pd.merge(df_date, df_tesouro_historico, on='data', how='left')
df_kpi = df_kpi.groupby('data').agg({'qt':'sum', 'qt_acum':'sum',  'vl_atualizado':'sum'}).reset_index()
df_kpi['vl_atualizado'] = np.where(df_kpi['qt_acum'] != 0, df_kpi['vl_atualizado'], np.nan)
df_kpi['vl_atualizado'] = df_kpi['vl_atualizado'].fillna(method='ffill').fillna(0)
df_kpi.head()

In [ ]:
# Cálculo dos aportes históricos.
vl_aporte = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                        (df_tesouro_historico['vl_total'] > 0) &
                                        (df_tesouro_historico['data'] <= date_interval[1]), 'vl_total'].sum()

vl_aporte_delta = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                            (df_tesouro_historico['vl_total'] > 0) &
                                            (df_tesouro_historico['data'].between(date_interval[0], date_interval[1])), 'vl_total'].sum()

# Cálculo dos valores resgatados.
vl_resgate = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                        (df_tesouro_historico['vl_total'] < 0) &
                                        (df_tesouro_historico['data'] <= date_interval[1]), 'vl_total'].sum()

vl_resgate_delta = df_tesouro_historico.loc[(df_tesouro_historico['qt'] != 0) &
                                            (df_tesouro_historico['vl_total'] < 0) &
                                            (df_tesouro_historico['data'].between(date_interval[0], date_interval[1])), 'vl_total'].sum()

# Cálculo do valor patrimonial.
vl_patrimonio = df_kpi.loc[df_kpi['data'] == date_interval[1], 'vl_atualizado'].sum()
vl_patrimonio_delta = vl_patrimonio - df_kpi.loc[df_kpi['data'] == date_interval[0], 'vl_atualizado'].sum()

# Cálculo do rendimento.
rendimento_nominal = round((vl_patrimonio - vl_resgate - vl_aporte) / vl_aporte * 100 , 1)
if  vl_aporte_delta != 0:
    rendimento_nominal_delta = round((vl_patrimonio_delta - vl_resgate_delta - vl_aporte_delta) / vl_aporte_delta * 100 , 1)
else:
    rendimento_nominal_delta = 0

vl_aporte, vl_aporte_delta, vl_resgate, vl_resgate_delta, vl_patrimonio, vl_patrimonio_delta, rendimento_nominal, rendimento_nominal_delta

## Bolsa

In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
df_bolsa['data'] = pd.to_datetime(df_bolsa['data'], format='%Y-%m-%d')
df_bolsa.head()

In [ ]:
# Salvar lá encima!
def etl_bolsa_historic_price(list_ticker_b3: list, start_date: str, end_date: str) -> np.array:

    # Utilizando a api do yf
    list_ticker_yf = [i + '.SA' for i in list_ticker_b3]
    long_string = ' '.join(list_ticker_yf)

    yf_df = yf.download(long_string, start=start_date, end=end_date, group_by='column', actions=True, interval='1wk')
    yf_df_close = yf_df['Close'].reset_index().sort_values('Date', ascending=False).round(2).fillna(method='ffill')
    yf_df_split = yf_df['Stock Splits'].reset_index().sort_values('Date', ascending=False).cumsum().round(2).fillna(method='ffill').replace(0, 1)

    for i in list_ticker_yf:
        yf_df_close[i] = yf_df_close[i] * (yf_df_split[i])

    df_price = yf_df_close.sort_values('Date', ascending=True)

    # Ajustes na base
    df_price.columns = ['data'] + list(list_ticker_b3)    
    df_price['data'] = pd.to_datetime(df_price['data'])
    df_price = pd.melt(df_price, id_vars=['data'], value_vars=list(list_ticker_b3), var_name='ticker', value_name='preco')
    
    return df_price

# Passo 1: extrair dados atualizados de cada ticker.
df_bolsa_historico = etl_bolsa_historic_price(list_ticker_b3=df_bolsa['ticker'].sort_values().unique(),
                                              start_date=df_bolsa['data'].min(),
                                              end_date=df_bolsa['data'].max())
df_bolsa_historico.head()

In [ ]:
"""
# Experimentando como usar o stock split
start_date = df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3', 'data'].dt.date.min()
end_date = df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3', 'data'].dt.date.max()
long_string = 'B3SA3.SA ITSA4.SA'

#start_date
yf_df = yf.download(long_string, start=start_date, end=end_date, group_by='column', actions=True, interval='1wk')
yf_df_close = yf_df['Close'].reset_index().sort_values('Date', ascending=False).round(2).fillna(method='ffill')
yf_df_split = yf_df['Stock Splits'].reset_index().sort_values('Date', ascending=False).cumsum().round(2).fillna(method='ffill').replace(0, 1)

display(df_bolsa.loc[df_bolsa['ticker'] == 'B3SA3'])
#display(yf_df)
display(yf_df_close.sort_values('Date'))
display(yf_df_split.sort_index())

for i in ['B3SA3.SA']:
    yf_df_close[i] = yf_df_close[i] * (yf_df_split[i])

df_price = yf_df_close.sort_values('Date', ascending=True)
display(df_price)
"""

In [ ]:
# Passo 2: juntar a carteira com os preços históricos.
df_bolsa2 = pd.merge(df_bolsa_historico, df_bolsa, on=['data', 'ticker'], how='left').fillna(0)
df_bolsa2.head()

In [ ]:
# Passo 3: calcular quantidade acumulada
for i in df_bolsa['ticker'].sort_values().unique():
    df_bolsa2.loc[df_bolsa2['ticker'] == i, 'qt_acum'] = df_bolsa2.loc[df_bolsa2['ticker'] == i, 'qt'].cumsum(skipna=True)
    
df_bolsa2['vl_atualizado'] = df_bolsa2['preco'] * df_bolsa2['qt_acum']
df_bolsa_negociacao = df_bolsa2.loc[(df_bolsa2['qt'] != 0) | (df_bolsa2['qt_acum'] != 0)]

display(df_bolsa_negociacao.head(2))